In [11]:
import boto3
import json
from botocore.client import Config
from semantic_router import Route
import os
from semantic_router.encoders import CohereEncoder
from semantic_router import RouteLayer


In [12]:
knowledgeBaseId = '4V4UV8DXOX'
model_id ='anthropic.claude-3-haiku-20240307-v1:0'
os.environ["COHERE_API_KEY"] = "RdXsDWxhX7C8sk5gFkeNYvPbdEtZKIlD81C1NtKt"

bedrock_config = Config(connect_timeout=120, 
                        read_timeout=120, 
                        retries={'max_attempts': 0}
                       )
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config)

In [13]:
coding_assistant = Route(
    name="coding_assistant",
    utterances=[
        "Write a python code for s3 bucket uploads",
        "write python code",
        "write code",
        "javascript code"
    ],
)
software_architect = Route(
    name="software_architect",
    utterances=[
        "What are the 5 pillars of well architected framework",
        "what are the best practices",
        "How to configure AWS service",
        "Tell me about the security pillar",
    ],
)
routes = [software_architect, coding_assistant]


In [14]:
encoder = CohereEncoder()
rl = RouteLayer(encoder=encoder, routes=routes)

2024-04-16 15:24:15 INFO semantic_router.utils.logger Initializing RouteLayer


In [15]:
def software_architect(prompt):
    relevant_documents = bedrock_agent_client.retrieve(
    retrievalQuery= {
        'text': prompt
    },
    knowledgeBaseId=knowledgeBaseId,
    retrievalConfiguration= {
        'vectorSearchConfiguration': {
            'numberOfResults': 3 # will fetch top 3 documents which matches closely with the query.
        }
    }
    )

    context = ''
    for doc in relevant_documents['retrievalResults']:
        context += doc['content']['text']
        
    final_prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt}: {context}"
                    }
                ]
            }
        ]
    }
    final_prompt = json.dumps(final_prompt)
    streaming_response = bedrock_client.invoke_model_with_response_stream(
        modelId=model_id,
        body=final_prompt,
    )
    stream = streaming_response.get('body')

    # a little bit of magic to buffer some response chunks then print them
    acc = 0
    buff = ''
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                blob = json.loads(chunk.get('bytes').decode())
                if blob['type'] == 'content_block_delta':
                    buff = buff + blob['delta']['text']
                    if acc < 20:
                        acc = acc +1
                    else:
                        print(buff)
                        buff=''
                        acc = 0

In [16]:
def coding_assistant(prompt):
    final_prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt}"
                    }
                ]
            }
        ]
    }
    final_prompt = json.dumps(final_prompt)
    streaming_response = bedrock_client.invoke_model_with_response_stream(
        modelId=model_id,
        body=final_prompt,
    )
    
    stream = streaming_response.get('body')

    # a little bit of magic to buffer some response chunks then print them
    acc = 0
    buff = ''
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                blob = json.loads(chunk.get('bytes').decode())
                if blob['type'] == 'content_block_delta':
                    buff = buff + blob['delta']['text']
                    if acc < 20:
                        acc = acc +1
                    else:
                        print(buff)
                        buff=''
                        acc = 0

In [18]:
prompt = 'pillars of well architected framework'
router = rl(prompt).name

if router == "software_architect":
    software_architect(prompt)
elif router == "coding_assistant":
    coding_assistant(prompt)

The key points from the passage are:

1. The AWS Well-Architected Framework is a set of
 best practices and guidelines developed by the AWS community of principal engineers to help customers build well-architected systems
 in the cloud.

2. The framework identifies 5 pillars of a well-architected system
: operational excellence, security, reliability, performance efficiency, and cost optimization.

3. The framework provides general
 design principles such as "stop guessing capacity needs" to help guide good design in the cloud.

4
. The framework is implemented through a structured review process where teams can assess their architectures against the framework's
 guidelines.

5. By carrying out multiple reviews across an organization, common thematic issues can be identified,
 which can then be addressed through mechanisms, training, or sharing of principal engineering knowledge.

6. The goal
 is to help organizations build reliable, secure, efficient, and cost-effective systems in the